In [ ]:
!pip install pyproj
import pyproj
from osgeo import gdal, ogr
import numpy as np


# Get the PROJ data directory
proj_data_dir = pyproj.datadir.get_data_dir()
print("PROJ data directory:", proj_data_dir)

In [ ]:
print("PROJ_LIB is set to:", os.environ['PROJ_LIB'])

In [ ]:
import os
os.environ['PROJ_LIB'] = '/opt/conda/lib/python3.10/site-packages/pyproj/proj_dir/share/proj'
os.environ['GDAL_DATA'] = '/opt/conda/lib/python3.10/site-packages/pyproj/proj_dir/share'
base_dir = os.getcwd()
print(base_dir)

In [ ]:
from osgeo import gdal, ogr
import numpy as np
#!unzip /home/jovyan/dataset_ex_2.zip

# Define pixel values for each feature
pixel_values = {
    'road_bound': [0, 0, 200],
    'buildings': [255, 0, 0],
    'road_markings': {
        'broken_line': [0, 20, 10],
        'cycle_lane': [0, 40, 0],
        'dashed_line': [0, 45, 70],
        'pedestrian_crossing': [0, 100, 0],
        'solid_line': [0, 45, 0],
        'stop_line': [0, 85, 0]
    }
}


In [ ]:
# Load vector data
road_bound_ds = ogr.Open('/home/jovyan/dataset_ex_2/grid-111/dop20rgb_386_5826_2022_bounds_grid-111.shp')
buildings_ds = ogr.Open('/home/jovyan/dataset_ex_2/grid-111/dop20rgb_386_5826_2022_buildings_grid-111.shp')
road_markings_ds = ogr.Open('/home/jovyan/dataset_ex_2/grid-111/dop20rgb_386_5826_2022_markings_grid-111.shp')

In [ ]:
gdal.SetConfigOption("GTIFF_SRS_SOURCE", "EPSG")
gdal.SetConfigOption('PROJ_LIB', '/opt/conda/lib/python3.10/site-packages/pyproj/proj_dir/share/proj')
raster_path = '/home/jovyan/dataset_ex_2/dop20rgb_386_5826_2022_grid_111.tif'
raster_ds = gdal.Open(raster_path)
geo_transform = raster_ds.GetGeoTransform()
projection = raster_ds.GetProjection()
cols = raster_ds.RasterXSize
rows = raster_ds.RasterYSize

# Create an empty RGB raster
driver = gdal.GetDriverByName('GTiff')
output_path = '/home/jovyan/dataset_ex_2/output_rgb_mask.tif'
output_ds = driver.Create(output_path, cols, rows, 3, gdal.GDT_Byte)
output_ds.SetGeoTransform(geo_transform)
output_ds.SetProjection(projection)

# Initialize the final RGB mask array
final_rgb_array = np.zeros((3, rows, cols), dtype=np.uint8)

# Function to rasterize a vector layer into the RGB mask
def rasterize_vector_layer(vector_ds, pixel_value):
    mem_drv = gdal.GetDriverByName('MEM')
    mem_ds = mem_drv.Create('', cols, rows, 1, gdal.GDT_Byte)
    mem_ds.SetGeoTransform(geo_transform)
    mem_ds.SetProjection(projection)
    
    gdal.RasterizeLayer(mem_ds, [1], vector_ds.GetLayer(), burn_values=[1])
    
    array = mem_ds.ReadAsArray()
    mask = array == 1
    rgb_array = np.zeros((3, rows, cols), dtype=np.uint8)
    rgb_array[0][mask] = pixel_value[0]
    rgb_array[1][mask] = pixel_value[1]
    rgb_array[2][mask] = pixel_value[2]
    
    return rgb_array

# Rasterize road bounds
road_bound_array = rasterize_vector_layer(road_bound_ds, pixel_values['road_bound'])
final_rgb_array += road_bound_array

# Rasterize buildings
buildings_array = rasterize_vector_layer(buildings_ds, pixel_values['buildings'])
final_rgb_array += buildings_array

# Rasterize road markings
for marking_type, pixel_value in pixel_values['road_markings'].items():
    marking_array = rasterize_vector_layer(road_markings_ds, pixel_value)
    final_rgb_array += marking_array

# Write the combined RGB array to the output raster bands
for band_idx in range(3):
    band = output_ds.GetRasterBand(band_idx + 1)
    band.WriteArray(final_rgb_array[band_idx])

# Close the output raster dataset
output_ds = None

print(f"RGB mask created and saved to {output_path}")